In [1]:
# @Time   : 2022/3/11
# @Author : Zihan Lin
# @Email  : zhlin@ruc.edu.cn

import argparse

from recbole_cdr.quick_start import run_recbole_cdr


parser = argparse.ArgumentParser()
parser.add_argument('--model', '-m', type=str, default='SSCDR', help='name of models')
parser.add_argument('--config_files', type=str, default=None, help='config files')

args, _ = parser.parse_known_args()

config_file_list = args.config_files.strip().split(' ') if args.config_files else None
run_recbole_cdr(model=args.model, config_file_list=config_file_list)


2024-05-09 01:56:54.457991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 01:56:54.657934: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 01:56:55.289524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-05-09 01:56:55.289610: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
import pandas as pd

digital_music = pd.read_csv('/home/hadh2/projects/cross_domain/data/Amazon/AmazonDigitalMusic/AmazonDigitalMusic.inter', sep='\t')
video_games = pd.read_csv('/home/hadh2/projects/cross_domain/data/Amazon/AmazonVideoGames/AmazonVideoGames.inter', sep='\t')
print(f'Digital Music len: {len(digital_music)}')
print(f'Video Games len: {len(video_games)}')


Digital Music len: 836006
Video Games len: 1324753


In [5]:
digital_music

,user_id:token,item_id:token,rating:float,timestamp:float
0,A2EFCYXHNK06IS,5555991584,5.0,978480000
1,A1WR23ER5HMAA9,5555991584,5.0,953424000
2,A2IR4Q0GPAFJKW,5555991584,4.0,1393545600
3,A2V0KUVAB9HSYO,5555991584,4.0,966124800
4,A1J0GL9HCA7ELW,5555991584,5.0,1007683200
...,...,...,...,...
836001,A38NRGVVEF3950,B00LWJDZHI,5.0,1405641600
836002,A36ZEAXJ5HMQXA,B00LWJDZHI,5.0,1406073600
836003,A1840L5R4OPDE4,B00LWJDZHI,5.0,1405900800
836004,A1HGHLLGGGAQZX,B00LWJDZHI,5.0,1405814400


In [4]:
digital_music = digital_music[digital_music['rating:float'] >= 3.0]
video_games = video_games[video_games['rating:float'] >= 3.0]
print(f'Digital Music len: {len(digital_music)}')
print(f'Video Games len: {len(video_games)}')

Digital Music len: 786273
Video Games len: 1094400


In [6]:
def retain(df):
    user_interaction_counts = df.groupby('user_id:token').size()

    # Lọc ra các user có số lượng interaction ít hơn 5
    users_to_remove = user_interaction_counts[user_interaction_counts < 5].index

    # Loại bỏ các user này khỏi DataFrame
    df = df[~df['user_id:token'].isin(users_to_remove)]

    # Nhóm theo item_id và tính số lượng interaction của mỗi item
    item_interaction_counts = digital_music.groupby('item_id:token').size()

    # Lọc ra các item có số lượng interaction ít hơn 5
    items_to_remove = item_interaction_counts[item_interaction_counts < 5].index

    # Loại bỏ các item này khỏi DataFrame
    df = df[~df['user_id:token'].isin(items_to_remove)]

    return df

digital_music = retain(digital_music)
video_games = retain(video_games)
print(f'Digital Music len: {len(digital_music)}')
print(f'Video Games len: {len(video_games)}')

Digital Music len: 222017
Video Games len: 240489


In [7]:
digital_music_users = set(digital_music['user_id:token'].to_list())
video_games_users = set(video_games['user_id:token'].to_list())
print(f'Digital Music num users: {len(digital_music_users)}')
print(f'Video Games num users: {len(video_games_users)}')

Digital Music num users: 21454
Video Games num users: 25098


In [8]:
len(digital_music_users.intersection(video_games_users))

988

In [10]:
digital_music_items = set(digital_music['item_id:token'].to_list())
video_games_items = set(video_games['item_id:token'].to_list())
print(f'Digital Music num items: {len(digital_music_items)}')
print(f'Video Games num items: {len(video_games_items)}')
len(digital_music_items.intersection(video_games_items))

Digital Music num items: 110851
Video Games num items: 30379


0

In [12]:
import pandas as pd

ml_1m = pd.read_csv('/home/hadh2/projects/cross_domain/RecBole-CDR/recbole_cdr/dataset_example/ml-1m/ml-1m.inter', sep='\t')
ml_100k = pd.read_csv('/home/hadh2/projects/cross_domain/RecBole-CDR/recbole_cdr/dataset_example/ml-100k/ml-100k.inter', sep='\t')
ml_1m_users = ml_1m['user_id:token'].apply(lambda x: x.replace('1m_', '')).to_list()
ml_100k_users = ml_100k['user_id:token'].apply(lambda x: x.replace('100k_', '')).to_list()
ml_1m_users = set(ml_1m_users)
ml_100k_users = set(ml_100k_users)
len(ml_1m_users.intersection(ml_100k_users))

943